In [1]:
import os
import pickle
import gc

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import L1L2

In [2]:
MODEL = "encoder-decoder"

config = {
    "N_PREV": 60,
    "N_FWD": 30,
    
    "ENCODER_SIZE": 16,
    "DECODER_SIZE": 16,
    
    "LR": 0.001,
    "BATCH_SIZE": 64,
    "EPOCHS": 30,
}

In [3]:
import wandb
from wandb.keras import WandbCallback
from secrets import WANDB
wandb.login(key=WANDB)

wandb: Currently logged in as: kmcguigan (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\kiern/.netrc


True

## **Load Data Function**

In [4]:
def get_data(fold):
    with open(f'./data/fold{fold}_data_v2.npy', mode='rb') as f:
        X_train = np.load(f)
        y_train = np.load(f)
        X_val = np.load(f)
        y_val = np.load(f)
    # with open(f'./data/fold{fold}_normalizer_v2.pkl', mode='rb') as f:
        # normalizer = pickle.load(f)
    return X_train, y_train, X_val, y_val, None

## **Model Functions**

In [5]:
def simple_encoder_decoder():
    inputs = tf.keras.layers.Input(shape=(config["N_PREV"], 1))
    x, hidden_state, cell_state = tf.keras.layers.LSTM(config["ENCODER_SIZE"], 
                                                       return_sequences=True, 
                                                       return_state=True, 
                                                       recurrent_initializer='glorot_uniform', 
                                                       activity_regularizer=L1L2(l1=0.000001, l2=0.000001))(inputs)
    
    decoder = tf.keras.layers.LSTM(config["DECODER_SIZE"],
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform', 
                                   activity_regularizer=L1L2(l1=0.000001, l2=0.000001))
    dropout = tf.keras.layers.Dropout(0.2)
    decoder_output = tf.keras.layers.Dense(1)
    all_outputs = []
    last_value = tf.expand_dims(inputs[:, -1, 0:1], 1)
    states = [hidden_state, cell_state]
    for i in range(config["N_FWD"]):
        x, hidden_state, cell_state = decoder(last_value, initial_state=states)
        states=[hidden_state, cell_state]
        x = dropout(x)
        x = decoder_output(x)
        last_value = x
        all_outputs.append(x)
    outputs = tf.keras.layers.Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(loss="mse", metrics=["mae"], optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"]))
    return model

## **Train Loop**

In [6]:
def train_model(model, X_train, y_train, X_val, y_val):
    reducer = tf.keras.callbacks.ReduceLROnPlateau(monior='val_loss', factor=0.1, patience=2, mode='min', cooldown=1)
    stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, mode='min', restore_best_weights=True)
    model.fit(X_train, y_train,
              batch_size=config["BATCH_SIZE"],
              epochs=config["EPOCHS"], 
              callbacks=[reducer, stopper, WandbCallback()],
              validation_data=(X_val, y_val),
              validation_batch_size=config["BATCH_SIZE"],
              shuffle=True)
    return model

In [7]:
def train():
    for fold in range(1, 4):
        run = wandb.init(project="time-series-methods", entity="kmcguigan", group=f"{MODEL}-model", config=config, job_type="train")
        run.name = f'{MODEL}-fold-{fold}'
        X_train, y_train, X_val, y_val, _ = get_data(fold)
        model = simple_encoder_decoder()
        model = train_model(model, X_train, y_train, X_val, y_val)
        run.finish()
        del model
        del X_train
        del y_train
        del X_val
        del y_val
        gc.collect()
    return

In [8]:
train()

Epoch 1/30
2485/2485 [==============================] - 165s 43ms/step - loss: 0.3860 - mae: 0.4777 - val_loss: 0.2423 - val_mae: 0.3759 - lr: 0.0010 - _timestamp: 1648938293.0000 - _runtime: 180.0000
Epoch 2/30
2485/2485 [==============================] - 91s 37ms/step - loss: 0.2326 - mae: 0.3670 - val_loss: 0.1786 - val_mae: 0.3212 - lr: 0.0010 - _timestamp: 1648938384.0000 - _runtime: 271.0000
Epoch 3/30
2485/2485 [==============================] - 95s 38ms/step - loss: 0.1913 - mae: 0.3290 - val_loss: 0.1435 - val_mae: 0.2835 - lr: 0.0010 - _timestamp: 1648938479.0000 - _runtime: 366.0000
Epoch 4/30
2485/2485 [==============================] - 93s 37ms/step - loss: 0.1673 - mae: 0.3061 - val_loss: 0.1371 - val_mae: 0.2746 - lr: 0.0010 - _timestamp: 1648938572.0000 - _runtime: 459.0000
Epoch 5/30
2485/2485 [==============================] - 93s 38ms/step - loss: 0.1710 - mae: 0.3104 - val_loss: 0.4537 - val_mae: 0.5202 - lr: 0.0010 - _timestamp: 1648938665.0000 - _runtime: 552.0000

epoch,▁▂▃▄▅▆▇█
loss,█▃▂▁▁▃▁▁
lr,██████▁▁
mae,█▃▂▁▁▃▂▁
val_loss,▃▂▁▁█▂▁▁
val_mae,▄▂▁▁█▂▁▁
best_epoch,3
best_val_loss,0.13714
epoch,7
loss,0.1761
lr,0.0001


Epoch 1/30
3163/3163 [==============================] - 191s 44ms/step - loss: 0.3174 - mae: 0.4269 - val_loss: 0.1768 - val_mae: 0.3126 - lr: 0.0010 - _timestamp: 1648939168.0000 - _runtime: 204.0000
Epoch 2/30
3163/3163 [==============================] - 157s 50ms/step - loss: 0.1897 - mae: 0.3296 - val_loss: 0.1475 - val_mae: 0.2798 - lr: 0.0010 - _timestamp: 1648939325.0000 - _runtime: 361.0000
Epoch 3/30
3163/3163 [==============================] - 131s 41ms/step - loss: 0.1672 - mae: 0.3073 - val_loss: 0.1338 - val_mae: 0.2634 - lr: 0.0010 - _timestamp: 1648939456.0000 - _runtime: 492.0000
Epoch 4/30
3163/3163 [==============================] - 120s 38ms/step - loss: 0.1646 - mae: 0.3040 - val_loss: 0.1390 - val_mae: 0.2684 - lr: 0.0010 - _timestamp: 1648939577.0000 - _runtime: 613.0000
Epoch 5/30
3163/3163 [==============================] - 124s 39ms/step - loss: 0.1640 - mae: 0.3033 - val_loss: 0.1544 - val_mae: 0.2865 - lr: 0.0010 - _timestamp: 1648939701.0000 - _runtime: 737.

epoch,▁▂▃▅▆▇█
loss,█▂▁▁▁▁▁
lr,█████▁▁
mae,█▃▂▁▁▁▁
val_loss,█▃▁▂▄▃▁
val_mae,█▃▁▂▄▃▂
best_epoch,2
best_val_loss,0.13383
epoch,6
loss,0.15678
lr,0.0001


Epoch 1/30
3841/3841 [==============================] - 227s 45ms/step - loss: 0.3373 - mae: 0.4458 - val_loss: 0.1792 - val_mae: 0.3119 - lr: 0.0010 - _timestamp: 1648940247.0000 - _runtime: 242.0000
Epoch 2/30
3841/3841 [==============================] - 164s 43ms/step - loss: 0.1878 - mae: 0.3267 - val_loss: 0.1419 - val_mae: 0.2738 - lr: 0.0010 - _timestamp: 1648940411.0000 - _runtime: 406.0000
Epoch 3/30
3841/3841 [==============================] - 166s 43ms/step - loss: 0.1767 - mae: 0.3166 - val_loss: 0.1376 - val_mae: 0.2674 - lr: 0.0010 - _timestamp: 1648940577.0000 - _runtime: 572.0000
Epoch 4/30
3841/3841 [==============================] - 176s 46ms/step - loss: 0.1881 - mae: 0.3264 - val_loss: 0.1395 - val_mae: 0.2716 - lr: 0.0010 - _timestamp: 1648940753.0000 - _runtime: 748.0000
Epoch 5/30
3841/3841 [==============================] - 182s 47ms/step - loss: 0.1706 - mae: 0.3108 - val_loss: 0.1193 - val_mae: 0.2483 - lr: 0.0010 - _timestamp: 1648940934.0000 - _runtime: 929.

epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
loss,█▃▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁
lr,███████▂▂▂▂▂▂▂▂▂▁▁
mae,█▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▄▄▂▅▃▂▂▁▁▁▁▁▁▁▁▁
val_mae,█▄▄▄▂▅▃▂▂▁▁▁▁▁▁▁▁▁
best_epoch,13
best_val_loss,0.11109
epoch,17
loss,0.13937
lr,1e-05
